In [2]:
import duckdb

In [3]:
# con = duckdb.connect("database/test.db")
duck_df = duckdb.read_csv("data/athens_data.csv")
type(duck_df)

duckdb.duckdb.DuckDBPyRelation

In [4]:
duckdb.execute(
    """
        PRAGMA enable_profiling;
        -- PRAGMA enable_profiling = 'query_tree';
        -- PRAGMA profiling_output = './profiling.json';

    """
)

In [5]:
# duckdb.sql("SELECT current_setting('profiling_output')")

In [6]:
duckdb.execute("PRAGMA show_tables;").fetchall()

[]

In [20]:
# %%timeit
duckdb.sql(
    """
        SELECT 
            Date AS date,
            station_name, 
            Temp AS temp, 
            "PM2.5" AS pm2_5,
        FROM duck_df 
        WHERE 
            Temp BETWEEN 10 AND 12
            AND Date BETWEEN DATE '2021-01-01' AND DATE '2021-01-31'
        ORDER BY Temp
        LIMIT 5
    """
)

┌─────────────────────┬──────────────────────────────┬────────────────────┬───────────────────┐
│        date         │         station_name         │        temp        │       pm2_5       │
│      timestamp      │           varchar            │       double       │      double       │
├─────────────────────┼──────────────────────────────┼────────────────────┼───────────────────┤
│ 2021-01-08 00:00:00 │ PANACEA_001                  │ 10.000026105949011 │              34.0 │
│ 2021-01-25 23:00:00 │ PANACEA_000                  │ 10.000708271896553 │            8.1875 │
│ 2021-01-26 21:00:00 │ Athens CAMS station 1        │ 10.000905261684789 │         2.1124547 │
│ 2021-01-25 23:00:00 │ CleanAir in Greece - Thiseio │ 10.000995675604765 │ 8.657692307692308 │
│ 2021-01-29 19:00:00 │ PARASKEVI                    │ 10.001125804259836 │               8.0 │
└─────────────────────┴──────────────────────────────┴────────────────────┴───────────────────┘